In [1]:
import sympy as sp
from IPython.display import display, Math

def math(t):
    return display(Math(t))


# Exercício Individual 4
<!-- [Enunciado](img/exercicio_individual_4/enunciado.png) -->

<img src='img/exercicio_individual_4/enunciado.png' height="50%" width="50%"/>


## Calculo analítico

Primeiro, vamos configurar os dados da questão:

In [2]:
# Valores que queremos encontrar
# s_long, s_hoop, s_r, e_r = sp.symbols("sigma_long, sigma_hoop, sigma_r, epsilon_r")

# Variáveis conhecidas do enunciado
R, h, t, r, P = sp.symbols("R, h, t, r, P")
# Módulo de Young, para calcular deformação
E, nu = sp.symbols("E, nu")

# Valores das variáveis conhecidas do enunciado (ver Opção)
opcao = {  # Usar unidades consistentes; Aqui, vamos usar mm e MPa
    R: 160,
    h: 280,
    t: sp.Rational(55, 10),  # fração ao invés de float para resultados exatos
    r: 10,
    P: 6,
    E: int(2e5),  # Ansys Structural Steel
    nu: sp.Rational(3, 10),  # Ansys Structural Steel
}


Para calcular as tensões, vamos utilizar as equações de vaso de pressão com paredes finas retiradas do livro:

$$
\sigma_{hoop} = \frac{P r}{t}
$$
$$
\sigma_{long} = \frac{P r}{2 t}
$$
$$
\sigma_{radi} = P
$$
$$
\epsilon_{hoop} = \frac{\sigma_{hoop}}{E} (1 - \frac{\nu}{2})
$$

É importante notar, entretanto, que a derivação dessas fórmulas utilizou o valor do raio interno do vaso (equivalente à área na qual a pressão é aplicada). Em outras palavras, no nosso problema, $r = R - t$

In [3]:
s_hoop = P * (R - t) / t
math(
    r"\sigma_{hoop}"
    r" = "
    f"{sp.latex(s_hoop)}"
    r" = "
    f"{sp.latex(s_hoop.subs(opcao))}"
    r" = "
    f"{sp.latex(s_hoop.evalf(5, subs=opcao))}"
    r"\,\mathrm{MPa}"
)


<IPython.core.display.Math object>

In [4]:
s_long = P * (R - t) / (2 * t)
math(
    r"\sigma_{long}"
    r" = "
    f"{sp.latex(s_long)}"
    r" = "
    f"{sp.latex(s_long.subs(opcao))}"
    r" = "
    f"{sp.latex(s_long.evalf(5, subs=opcao))}"
    r"\,\mathrm{MPa}"
)


<IPython.core.display.Math object>

In [5]:
s_radi = P
math(
    r"\sigma_{radi}"
    r" = "
    f"{sp.latex(s_radi)}"
    r" = "
    f"{sp.latex(s_radi.subs(opcao))}"
    r" = "
    f"{sp.latex(s_radi.evalf(5, subs=opcao))}"
    r"\,\mathrm{MPa}"
)


<IPython.core.display.Math object>

In [6]:
e_hoop = s_hoop / E * (1 - nu / 2)
math(
    r"\epsilon_{hoop}"
    r" = "
    f"{sp.latex(e_hoop)}"
    r" = "
    f"{sp.latex(e_hoop.subs(opcao))}"
    r" = "
    f"{sp.latex(e_hoop.evalf(5, subs=opcao))}"
)


<IPython.core.display.Math object>

## Cálculo Numérico (Ansys)
Podemos também calcular os mesmos valores utilizando o método dos elementos finitos. Aqui vamos realizar esse cálculo com a ajuda do software Ansys.

Primeiramente, precisamos criar a geometria do problema. Aqui, vamos aproveitar as simetrias da geometria para diminuir o tamanho do problema; assim, dividimos a geometria ao longo dos 3 planos de simetria (escolhendo 2 planos de simetria radial perpendiculares). A geometria é mostrada na figura abaixo, e as dimensões foram configuradas de acordo com o desenho do enunciado da questão e a Opção V:

$$
    R = 160 \,\mathrm{mm} \\
    h = 280 \mathrm{mm} \\
    t = 5.5  \,\mathrm{MPa} \\
    r = 10 \,\mathrm{mm} \\
    P = 6 \,\mathrm{MPa}
$$

<!-- ![Geometria](img/exercicio_individual_4/geometria.png) -->
<img src='img/exercicio_individual_4/geometria.png' height="50%" width="50%"/>


Então, utilizamos a geometria para gerar uma malha de elementos finitos. Na figura abaixo, podemos ver a malha resultante colorida pelo indicador Element Quality do software. Foram utilizadas as configurações de Face Meshing nas faces cilíndricas e Body Sizing (com tamanho igual à espessura $t$ ) para gerar a malha automaticamente.
<!-- ![Malha](img/exercicio_individual_4/malha.png) -->

<img src='img/exercicio_individual_4/malha.png' height="50%" width="50%"/>


Com a malha pronta, vamos aplicar as condições de contorno para que o software resolva o problema. Precisamos aplicar as condições de simetria nas faces de corte, representando a impossibilidade de deformação ao longo das faces de simetria. No software, fazemos isso com a condição de Frictionless Support. Também precisamos aplicar a condição de pressão nas faces internas do vaso. Para isso, utilizamos a ferramenta Pressure. as condições aplicadas, então, podem ser vistas na figura abaixo (a condição de Frictionless Support também está aplicada na face de simetria inferior, não visível na figura).

<!-- ![Condições de contorno](img/exercicio_individual_4/condicoes_contorno.png) -->
<img src='img/exercicio_individual_4/condicoes_contorno.png' height="50%" width="50%"/>


O software então pode resolver o modelo para a malha e as condições de contorno configuradas. Para calcular os valores de tensões e de deformação relativa que queremos, precisamos investigar esses valores em um ponto específico da geometria/malha. Como queremos comparar nosso resultado com o método analítico das paredes finas, é interessante calcular esses valores ao longo da espessura $t$ do vaso; entretanto, esses valores não são constantes ao longo da espessura. Para obter um resultado mais comparável à teoria, então, vamos calcular os valores em 2 pontos, um na face interna e outro na face externa do vaso. Os pontos escolhidos podem ser vistos nas figuras abaixo. A fim de obter um resultado final para comparar à teoria, vamos então calcular a média desses valores, representando a aproximação de paredes finas que foi feita na teoria ($\sigma_{radi}$ não será avaliada em média, pois a teoria supõe uma função descontínua; $\sigma_{radi} = P$ na face interna e $\sigma_{radi} = 0$ na face externa). Os resultados foram tabelados abaixo.

<!-- ![Probe externa](img/exercicio_individual_4/probe_ext.png) ![Probe interna](img/exercicio_individual_4/probe_int.png) -->
<img src='img/exercicio_individual_4/probe_ext.png' height="49%" width="49%"/>
<img src='img/exercicio_individual_4/probe_int.png' height="49%" width="49%"/>



|Ponto   |$\sigma_{hoop}$ [MPa]|$\sigma_{long}$ [MPa]|$\sigma_{radi}$ [MPa]|$\epsilon_{hoop}$ [mm/mm]|
| ------ | ------------------: | ------------------: | ------------------: | ----------------------: |
|Exterior|              165.53 |              82.779 |            0.033008 |               7.0343E-4 |
|Interior|              171.82 |              82.859 |             -6.1554 |               7.4402E-4 |
|Média   |              168.67 |              82.819 |                   - |               7.2373E-4 |
|Teoria  |              168.55 |              84.273 |             -6,0000 |               7.1632E-4 |


Para analisar a qualidade do resultado numérico, podemos calcular a margem de erro entre os valores encontrados e os valores teóricos calculados. Vamos analizar esse erro de 2 formas, utilizando o valor médio (mais condizente com a suposição de paredes finas) e utilizando o erro máximo (uma comparação mais real da utilidade da simplificação). Na tabela abaixo podemos ver esses erros para cada medida:

|Erro    | \(\sigma_{hoop}\) | \(\sigma_{long}\) | \(\sigma_{radi}\) | \(\epsilon_{hoop}\) |
| ------ | ----------------: | ----------------: | ----------------: | ------------------: |
|Médio   |             0.071 |            -1.725 |                 - |               1.034 |
|Máximo  |             1.940 |            -1.773 |             2.590 |               3.867 |


Conforme pode ser visto na tabela de erros, os valores numéricos são extremamente próximos dos cálculos teóricos de paredes finas, validando nosso trabalho